In [6]:
import os
path = "/root/data/Datasets/IM_Small_ImageNet/train"
numbers = {}
for cls in os.listdir(path):
    numbers[cls] = len(os.listdir(os.path.join(path, cls)))


with open("dataset_info/Small_ImageNet.txt", "r") as f:
    lines = f.read().splitlines()
    small_imagenet_stats = dict()
    cls_ids, cls_idxs, cls_names = zip(*[i.split(" ") for i in lines])
    for cls_id, cls_idx, cls_name in zip(cls_ids, cls_idxs, cls_names):
        sample_dict = dict()
        sample_dict['name'] = cls_name
        sample_dict['id'] = cls_id
        sample_dict['idx'] = cls_idx
        sample_dict['numbers'] = numbers[cls_id]
        small_imagenet_stats[cls_idx] = sample_dict

with open("dataset_info/IM_Small_ImageNet.txt", "w") as f:
    for k, v in small_imagenet_stats.items():
        f.write(f"{v['id']} {v['idx']} {v['name']} {v['numbers']}\n")

In [25]:
import os

default_cmd = "python main.py --base configs/stable-diffusion/v1-finetune_test.yaml -t --actual_resume /home/jhshin4727/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt -n test --gpus 0, --init_word * "
main_dir = "/root/data/Datasets/Small_ImageNet/train"
log_dir = "ti_test"
port = 10000

for cls in os.listdir(main_dir):
    a = f"--data_root {os.path.join('/home/jhshin4727/data/IM_Small_ImageNet/train/', cls)} "
    b = f"--logdir {log_dir}"
    cmd = f"MASTER_PORT={port} " + default_cmd + a + b
    port += 1
    print(cmd)

MASTER_PORT=10000 python main.py --base configs/stable-diffusion/v1-finetune_test.yaml -t --actual_resume /home/jhshin4727/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt -n test --gpus 0, --init_word * --data_root /home/jhshin4727/data/IM_Small_ImageNet/train/n01677366 --logdir ti_test
MASTER_PORT=10001 python main.py --base configs/stable-diffusion/v1-finetune_test.yaml -t --actual_resume /home/jhshin4727/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt -n test --gpus 0, --init_word * --data_root /home/jhshin4727/data/IM_Small_ImageNet/train/n02236044 --logdir ti_test
MASTER_PORT=10002 python main.py --base configs/stable-diffusion/v1-finetune_test.yaml -t --actual_resume /home/jhshin4727/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt -n test --gpus 0, --init_word * --data_root /home/jhshin4727/data/IM_Small_ImageNet/train/n04254680 --logdir ti_test
MASTER_PORT=10003 python main.py --base configs/stable-diffusion/v1-finetune_test.yaml -t --actual_resume /home/jhshin4727/Stable-Diffusion-Ch

In [26]:
os.listdir(main_dir)

['n01677366',
 'n02236044',
 'n04254680',
 'n01910747',
 'n02422106',
 'n04355933',
 'n02783161',
 'n02113186',
 'n02423022',
 'n03995372',
 'n03770439',
 'n01667778',
 'n03814639',
 'n01728572',
 'n03876231',
 'n04264628',
 'n02113624',
 'n03216828',
 'n01930112',
 'n01632458',
 'n02137549',
 'n02167151',
 'n02117135',
 'n03400231',
 'n02102177',
 'n04153751',
 'n04442312',
 'n02279972',
 'n03662601',
 'n04591713',
 'n03956157',
 'n03902125',
 'n02815834',
 'n01950731',
 'n02877765',
 'n02504458',
 'n02174001',
 'n02111500',
 'n02111129',
 'n02971356',
 'n04465501',
 'n04548362',
 'n02074367',
 'n01872401',
 'n04258138',
 'n02807133',
 'n04285008',
 'n02206856',
 'n03384352',
 'n07932039',
 'n04065272',
 'n07753113',
 'n02093754',
 'n01917289',
 'n01560419',
 'n13054560',
 'n02641379',
 'n03063689',
 'n01494475',
 'n02859443',
 'n03345487',
 'n01740131',
 'n02106030',
 'n04131690',
 'n02364673',
 'n03028079',
 'n04554684',
 'n01806143',
 'n02100877',
 'n02692877',
 'n02667093',
 'n077

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from PIL import Image

cls_num = 100
imb_type = 'exp'
imb_factor = 0.005

def center_crop(img):
    width, height = img.size
    if width > height:
        left = (width - height) / 2
        top = 0
        right = (width + height) / 2
        bottom = height
    else:
        left = 0
        top = (height - width) / 2
        right = width
        bottom = (height + width) / 2
    return img.crop((left, top, right, bottom))

def get_img_num_per_cls(cls_num, imb_type, imb_factor):
    img_max = 1000
    img_num_per_cls = []
    if imb_type == 'exp':
        for cls_idx in range(cls_num):
            num = img_max * (imb_factor**(cls_idx / (cls_num - 1.0)))
            img_num_per_cls.append(int(num))
    return img_num_per_cls

img_num_per_cls = get_img_num_per_cls(cls_num, imb_type, imb_factor)

main_dir = "/root/data/Datasets/Small_ImageNet/train"
new_dir = "/root/data/Datasets/IM_Small_ImageNet/train"

for i, cls in enumerate(os.listdir(main_dir)):
    cls_dir = os.path.join(main_dir, cls)
    new_cls_dir = os.path.join(new_dir, cls)
    os.makedirs(new_cls_dir, exist_ok=True)
    for j, img in enumerate(os.listdir(cls_dir)):
        if j < img_num_per_cls[i]:
            image = Image.open(os.path.join(cls_dir, img)).convert('RGB')
            image = center_crop(image)
            image = image.resize((512, 512), Image.LANCZOS)
            image.save(os.path.join(new_cls_dir, img))
    print(f"Class {i} done with {len(os.listdir(new_cls_dir))} images")

Class 0 done with 1000 images
Class 1 done with 947 images
Class 2 done with 898 images
Class 3 done with 851 images
Class 4 done with 807 images
Class 5 done with 765 images
Class 6 done with 725 images
Class 7 done with 687 images
Class 8 done with 651 images
Class 9 done with 617 images
Class 10 done with 585 images
Class 11 done with 555 images
Class 12 done with 526 images
Class 13 done with 498 images
Class 14 done with 472 images
Class 15 done with 448 images
Class 16 done with 424 images
Class 17 done with 402 images
Class 18 done with 381 images
Class 19 done with 361 images
Class 20 done with 342 images
Class 21 done with 325 images
Class 22 done with 308 images
Class 23 done with 292 images
Class 24 done with 276 images
Class 25 done with 262 images
Class 26 done with 248 images
Class 27 done with 235 images
Class 28 done with 223 images
Class 29 done with 211 images
Class 30 done with 200 images
Class 31 done with 190 images
Class 32 done with 180 images
Class 33 done with 

In [18]:
os.listdir(main_dir)

['n01677366',
 'n02236044',
 'n04254680',
 'n01910747',
 'n02422106',
 'n04355933',
 'n02783161',
 'n02113186',
 'n02423022',
 'n03995372',
 'n03770439',
 'n01667778',
 'n03814639',
 'n01728572',
 'n03876231',
 'n04264628',
 'n02113624',
 'n03216828',
 'n01930112',
 'n01632458',
 'n02137549',
 'n02167151',
 'n02117135',
 'n03400231',
 'n02102177',
 'n04153751',
 'n04442312',
 'n02279972',
 'n03662601',
 'n04591713',
 'n03956157',
 'n03902125',
 'n02815834',
 'n01950731',
 'n02877765',
 'n02504458',
 'n02174001',
 'n02111500',
 'n02111129',
 'n02971356',
 'n04465501',
 'n04548362',
 'n02074367',
 'n01872401',
 'n04258138',
 'n02807133',
 'n04285008',
 'n02206856',
 'n03384352',
 'n07932039',
 'n04065272',
 'n07753113',
 'n02093754',
 'n01917289',
 'n01560419',
 'n13054560',
 'n02641379',
 'n03063689',
 'n01494475',
 'n02859443',
 'n03345487',
 'n01740131',
 'n02106030',
 'n04131690',
 'n02364673',
 'n03028079',
 'n04554684',
 'n01806143',
 'n02100877',
 'n02692877',
 'n02667093',
 'n077

In [14]:
import numpy as np
ai = []
ac = []
for i in range(10, 200):
    c = 1000 // i
    a = np.asarray(list(range(0, 1000, c)))

    if c not in ac and (999 not in a):
        ac.append(c)
        ai.append(i)
print(ai)
print(ac)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 41, 42, 44, 46, 48, 51, 53, 56, 59, 63, 67, 72, 77, 84, 91, 112, 126, 143, 167]
[100, 90, 83, 76, 71, 66, 62, 58, 55, 52, 50, 47, 45, 43, 41, 40, 38, 35, 34, 33, 32, 31, 30, 29, 28, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 8, 7, 6, 5]


In [12]:
c = 1000 // 116
a = np.asarray(list(range(0, 1000, c)))
a

array([  0,   8,  16,  24,  32,  40,  48,  56,  64,  72,  80,  88,  96,
       104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200,
       208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 304,
       312, 320, 328, 336, 344, 352, 360, 368, 376, 384, 392, 400, 408,
       416, 424, 432, 440, 448, 456, 464, 472, 480, 488, 496, 504, 512,
       520, 528, 536, 544, 552, 560, 568, 576, 584, 592, 600, 608, 616,
       624, 632, 640, 648, 656, 664, 672, 680, 688, 696, 704, 712, 720,
       728, 736, 744, 752, 760, 768, 776, 784, 792, 800, 808, 816, 824,
       832, 840, 848, 856, 864, 872, 880, 888, 896, 904, 912, 920, 928,
       936, 944, 952, 960, 968, 976, 984, 992])

In [10]:
from PIL import Image
from torchvision.utils import make_grid, save_image
from torchvision import transforms
import os
import random
import torch

def choose_random_25_and_save_as_grid(path, save_path):
    """Choose 25 random images from a folder and save them as a grid.
    
    Args:
        path: Path to folder containing images.
        save_path: Path to save the grid.
    """
    files = os.listdir(path)
    files = [os.path.join(path, file) for file in files]
    files = random.sample(files, 25)
    images = [Image.open(file).resize((128, 128)) for file in files]
    images = torch.stack([transforms.ToTensor()(image) for image in images])
    grid = make_grid(images, nrow=5)
    save_image(grid, save_path)

choose_random_25_and_save_as_grid("/root/code/sample_data/corncob/corncob", "real_corncob.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob-naive/samples", "corncob_naive.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob0001_12023-02-15T03-24-29_test/samples", "corncob_1.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob00012023-02-14T03-25-23_test/samples", "corncob_1_wrong.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob00052023-02-14T05-00-49_test/samples", "corncob_5.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob00102023-02-14T03-25-28_test/samples", "corncob_10.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob00502023-02-14T05-03-14_test/samples", "corncob_50.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob01002023-02-14T03-25-33_test/samples", "corncob_100.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob05002023-02-14T05-10-47_test/samples", "corncob_500.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob10002023-02-14T03-25-37_test/samples", "corncob_1000.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob_1000_mixed_randn/samples", "corncob_mixed_randn.png")
choose_random_25_and_save_as_grid("/root/code/textual_inversion/outputs/corncob_1000_mixed/samples", "corncob_mixed_uni.png")



ValueError: unknown file extension: 

In [4]:
import os
import shutil

path = "outputs/"
projects = os.listdir(path)
for project in projects:
    path2 = path + project + "/"
    if os.path.exists(os.path.join(path2, "a-photo-of-*-0000.jpg")):
        os.remove(os.path.join(path2, "a-photo-of-*-0000.jpg"))
    if os.path.exists(os.path.join(path2, "samples", ".ipynb_checkpoints")):
        shutil.rmtree(os.path.join(path2, "samples", ".ipynb_checkpoints"))


In [15]:
import os
default_command = 'python scripts/stable_txt2img.py --n_samples 4 --n_iter 128  --ckpt /root/code/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt --prompt "a photo of *"'

paths = os.listdir("/root/code/textual_inversion/logs")
paths = sorted([i for i in paths if ("_12023" in i or "_22023" in i or "_32023" in i or "_42023" in i) and "0250_" not in i])
paths.remove("corncob0001_12023-02-15T03-24-29_test")
for i, path in enumerate(paths):
    cmd = default_command + f' --embedding_path logs/{path}/checkpoints/embeddings_gs-9999.pt --outdir outputs/{paths[i][7:11]}/{paths[i][:13]}'
    print(cmd)


'0001'

In [8]:
default_command = "python main.py -t --actual_resume /root/code/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt -n test --init_word *"

k = 0
for j in range(1,5):
    for i in [1, 5, 10, 50, 100, 500, 1000]:
        data_root = f"/root/code/sample_data/sub_corncobs/{i}/corncob{i:04d}_{j}"
        base = f"configs/stable-diffusion/v1-finetune_corncob{i:04d}.yaml"
        gpus = f"--gpus 0,"
        command = f"MASTER_PORT={12345+k} {default_command} --data_root {data_root} --base {base} {gpus}"
        print(command)
        k += 1

MASTER_PORT=12345 python main.py -t --actual_resume /root/code/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt -n test --init_word * --data_root /root/code/sample_data/sub_corncobs/1/corncob0001_1 --base configs/stable-diffusion/v1-finetune_corncob0001.yaml --gpus 0,
MASTER_PORT=12346 python main.py -t --actual_resume /root/code/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt -n test --init_word * --data_root /root/code/sample_data/sub_corncobs/5/corncob0005_1 --base configs/stable-diffusion/v1-finetune_corncob0005.yaml --gpus 0,
MASTER_PORT=12347 python main.py -t --actual_resume /root/code/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt -n test --init_word * --data_root /root/code/sample_data/sub_corncobs/10/corncob0010_1 --base configs/stable-diffusion/v1-finetune_corncob0010.yaml --gpus 0,
MASTER_PORT=12348 python main.py -t --actual_resume /root/code/Stable-Diffusion-Checkpoints/v1-5-pruned.ckpt -n test --init_word * --data_root /root/code/sample_data/sub_corncobs/50/corncob0050_1 --bas

In [3]:
import os
import numpy as np
import shutil

input_path = '/root/code/sample_data/corncob_imagenet_train_512/corncob'
files = [f for f in os.listdir(input_path) if f.endswith('.JPEG')]


for j in range(1, 5):
    for i in [1, 5, 10, 50, 100, 500, 1000]:
        random_files = []
        output_path = os.path.join(f'/root/code/sample_data/sub_corncobs/{i}', f'corncob{i:04d}_{j}')
        if os.path.exists(output_path):
            shutil.rmtree(output_path)
        os.makedirs(output_path, exist_ok=True)
        files = [f for f in files if f not in random_files]
        random_files += np.random.choice(files, i - len(random_files), replace=False).tolist()
        print(len(random_files))
        for f in random_files:
            shutil.copy(os.path.join(input_path, f), output_path)

1
5
10
50
100
500
1000
1
5
10
50
100
500
1000
1
5
10
50
100
500
1000
1
5
10
50
100
500
1000


In [17]:
i = [f for f in files[0::4] if f in files[3::4]]
i

[]

In [18]:
import os
import numpy as np
import shutil
import random

input_path = '/root/code/sample_data/custom_diffusion_data/corncob1000'
files = [f for f in os.listdir(input_path) if f.endswith('.JPEG')]
random.shuffle(files)

for i in range(4):
    output_path = os.path.join('/root/code/sample_data/custom_diffusion_data/', f'corncob1000_{i}')
    # shutil.rmtree(output_path)
    os.makedirs(output_path, exist_ok=True)
    for f in files[i::4]:
        shutil.copy(os.path.join(input_path, f), output_path)

In [3]:
from PIL import Image
import os

def center_crop(img):
    width, height = img.size
    if width > height:
        left = (width - height) / 2
        top = 0
        right = (width + height) / 2
        bottom = height
    else:
        left = 0
        top = (height - width) / 2
        right = width
        bottom = (height + width) / 2
    return img.crop((left, top, right, bottom))

def load_center_crop_resize_save(input_path, size, output_path):
    for img in os.listdir(input_path):
        img_name = img
        img = Image.open(os.path.join(input_path, img)).convert('RGB')
        img = center_crop(img)
        img = img.resize((size, size), Image.LANCZOS)
        img.save(os.path.join(output_path, img_name))

input_path = "/root/code/sample_data/corncob_valid"
output_path = "/root/code/sample_data/corncob_valid_512"
load_center_crop_resize_save(input_path, 512, output_path)

In [2]:
from torchvision.models import resnet50
import torch
from PIL import Image

image = Image.open("/root/data/data/Synthetic_ImageNet_Text_only/train/n01807496/fake_86_0646.JPEG")
resnet50 = resnet50(pretrained=True)


In [6]:
from transformers import AutoFeatureExtractor, ResNetForImageClassification
import torch
from PIL import Image

image = Image.open("/root/data/data/Synthetic_ImageNet_Text_only/train/n01807496/fake_86_0646.JPEG")
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

inputs = feature_extractor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

partridge
